In [20]:
from keras.models import Sequential
from keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense, RandomFlip, RandomRotation
import os
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from keras.optimizers import Adam
from keras._tf_keras.keras.preprocessing.image import ImageDataGenerator
import random
from skimage import io, transform, exposure
from sklearn.metrics import classification_report

### Load data V2:

In [2]:
csv_path_train = r"D:\Faculty materials\bachelors\datasets\GTSRB\Train.csv"
csv_path_test = r"D:\Faculty materials\bachelors\datasets\GTSRB\Test.csv"
dataset_path = r"D:\Faculty materials\bachelors\datasets\GTSRB"

def load_data(csv_path):
    image_list = []
    labels_list = []
    classes = 43

    rows = open(csv_path).read().strip().split("\n")[1:]
    random.shuffle(rows)

    for (i, row) in enumerate(rows):
        if i > 0 and i % 1000 == 0:
            print(f"[INFO] processed {i} images...")
        (label, image_path) = row.strip().split(",")[-2:]
        image_path = os.path.join(dataset_path, image_path)
        image = io.imread(image_path)
        image = transform.resize(image, (45, 45))
        # image = exposure.equalize_adapthist(image, clip_limit=0.1)
        image_list.append(image)
        labels_list.append(label)

    image_list = np.array(image_list, dtype=np.float32) / 255.0
    labels_list = np.array(labels_list)
    return image_list, labels_list


In [3]:
print("[INFO] loading train and validation images...")
image_list, labels_list = load_data(csv_path_train)
x_train, x_val, y_train, y_val = train_test_split(image_list, labels_list, test_size=0.2, random_state=42, shuffle=True)

print("[INFO] loading test images...")
x_test, y_test = load_data(csv_path_test)

y_train = to_categorical(y_train, 43)
y_val = to_categorical(y_val, 43)
y_test = to_categorical(y_test, 43)

total_class = y_train.sum(axis=0)
class_weights = dict()

for i in range(0, len(total_class)):
    class_weights[i] = total_class.max() / total_class[i]

[INFO] loading train and validation images...
[INFO] processed 1000 images...
[INFO] processed 2000 images...
[INFO] processed 3000 images...
[INFO] processed 4000 images...
[INFO] processed 5000 images...
[INFO] processed 6000 images...
[INFO] processed 7000 images...
[INFO] processed 8000 images...
[INFO] processed 9000 images...
[INFO] processed 10000 images...
[INFO] processed 11000 images...
[INFO] processed 12000 images...
[INFO] processed 13000 images...
[INFO] processed 14000 images...
[INFO] processed 15000 images...
[INFO] processed 16000 images...
[INFO] processed 17000 images...
[INFO] processed 18000 images...
[INFO] processed 19000 images...
[INFO] processed 20000 images...
[INFO] processed 21000 images...
[INFO] processed 22000 images...
[INFO] processed 23000 images...
[INFO] processed 24000 images...
[INFO] processed 25000 images...
[INFO] processed 26000 images...
[INFO] processed 27000 images...
[INFO] processed 28000 images...
[INFO] processed 29000 images...
[INFO]

In [15]:
model = Sequential()
model.add(Conv2D(filters=24, kernel_size=(3,3), activation='relu', input_shape=x_train.shape[1:]))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(filters=24, kernel_size=(3,3), activation='relu'))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu'))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu'))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(filters=48, kernel_size=(3,3), activation='relu'))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(filters=48, kernel_size=(3,3), activation='relu'))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(43, activation='softmax'))

model.summary()


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_32 (Conv2D)              │ (None, 43, 43, 24)     │           672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 43, 43, 24)     │            96 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_33 (Conv2D)              │ (None, 41, 41, 24)     │         5,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 41, 41, 24)     │            96 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_16 (MaxPooling2D) │ (None, 20, 20, 24)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_34 (Conv2D)              │ (None, 18, 18, 32)     │         6,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 18, 18, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_35 (Conv2D)              │ (None, 16, 16, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 16, 16, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_17 (MaxPooling2D) │ (None, 8, 8, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_36 (Conv2D)              │ (None, 6, 6, 48)       │        13,872 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 6, 6, 48)       │           192 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_37 (Conv2D)              │ (None, 4, 4, 48)       │        20,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 4, 4, 48)       │           192 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_18 (MaxPooling2D) │ (None, 2, 2, 48)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ (None, 192)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 512)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 43)             │        22,059 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 178,435 (697.01 KB)

 Trainable params: 178,019 (695.39 KB)

 Non-trainable params: 416 (1.62 KB)

In [16]:
BS = 32
EPOCH = 20

In [17]:
aug = ImageDataGenerator(rotation_range=10,
                         zoom_range=0.15,
                         width_shift_range=0.1,
                         height_shift_range=0.1,
                         shear_range=0.15,
                         horizontal_flip=False,
                         vertical_flip=False,
                         fill_mode='nearest')

In [18]:
print("[INFO] Compiling the model...")
optim = Adam(learning_rate=1e-3, decay=1e-3/5)
model.compile(optimizer=optim,
              loss='categorical_crossentropy', metrics=["accuracy"])

print("[INFO] Training the model...")
history = model.fit(aug.flow(x_train, y_train, batch_size=BS), validation_data=(x_val, y_val), epochs=EPOCH, class_weight=class_weights)
model.save("TSR_CNN2_first_try.h5")

[INFO] Compiling the model...
[INFO] Training the model...
Epoch 1/20
981/981 ━━━━━━━━━━━━━━━━━━━━ 83s 80ms/step - accuracy: 0.2644 - loss: 6.7671 - val_accuracy: 0.7467 - val_loss: 0.8315
Epoch 2/20
981/981 ━━━━━━━━━━━━━━━━━━━━ 82s 83ms/step - accuracy: 0.7836 - loss: 1.4246 - val_accuracy: 0.9653 - val_loss: 0.1108
Epoch 3/20
981/981 ━━━━━━━━━━━━━━━━━━━━ 82s 83ms/step - accuracy: 0.8948 - loss: 0.7347 - val_accuracy: 0.9375 - val_loss: 0.2133
Epoch 4/20
981/981 ━━━━━━━━━━━━━━━━━━━━ 83s 84ms/step - accuracy: 0.9321 - loss: 0.4652 - val_accuracy: 0.9607 - val_loss: 0.1256
Epoch 5/20
981/981 ━━━━━━━━━━━━━━━━━━━━ 89s 90ms/step - accuracy: 0.9421 - loss: 0.4082 - val_accuracy: 0.8196 - val_loss: 0.7293
Epoch 6/20
981/981 ━━━━━━━━━━━━━━━━━━━━ 103s 105ms/step - accuracy: 0.9497 - loss: 0.3586 - val_accuracy: 0.9827 - val_loss: 0.0517
Epoch 7/20
981/981 ━━━━━━━━━━━━━━━━━━━━ 81s 82ms/step - accuracy: 0.9613 - loss: 0.2808 - val_accuracy: 0.9772 - val_loss: 0.0760
Epoch 8/20
981/981 ━━━━━━━━━━

In [21]:
sign_labels = open("SignsNames.csv").read().strip().split("\n")[1:]
sign_labels = [l.split(",")[1] for l in sign_labels]

print("[INFO] evaluating the model...")
predictions = model.predict(x_test, batch_size=BS)
print(classification_report(y_test.argmax(axis=1), predictions.argmax(axis=1), target_names=sign_labels))

[INFO] evaluating the model...
395/395 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step
                                                            precision    recall  f1-score   support

                                      Speed limit (20km/h)       0.87      0.45      0.59        60
                                      Speed limit (30km/h)       0.95      0.48      0.64       720
                                      Speed limit (50km/h)       0.59      0.34      0.43       750
                                      Speed limit (60km/h)       0.08      0.85      0.14       450
                                      Speed limit (70km/h)       0.86      0.18      0.30       660
                                      Speed limit (80km/h)       0.79      0.17      0.27       630
                               End of speed limit (80km/h)       0.00      0.00      0.00       150
                                     Speed limit (100km/h)       1.00      0.04      0.08       450
                          

d:\Faculty materials\BACHELORS-THESIS\.venv\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Faculty materials\BACHELORS-THESIS\.venv\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Faculty materials\BACHELORS-THESIS\.venv\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} i